## HS 2022 export analysis

****

#### Background:

Goods codes which are categorised under the "Harmonised system" (HS) are digits broken down into 2,4 and 6 to uniquely identfiy goods consistently across the internatonal trading system.

These HS codes are chnaged at the 6-digit level every 5 years. The last change was in 2017 and the latest change has ocurred in 2022. 

There are approrixmately 500 tariff line (8-digit) goods codes which are affected by these HS chnages. 

To help determne codes and countries of interest which are most affected, analysis has been undertaken to determine the:

1. Top exported codes
2. Top countries these codes are exported to

### Outcome:

Once the major countries have been indeitified this will allow further indepth analysis of individual tariff schedules at the World Trade Organisationand how the  exports may be affected with the latest HS changes. 

***

### Analysis:

The 500 HS codes affected require UK export analysis covering 2018 to 2020 and 2021 following EU-exit. Outputs will determine the largest UK exports, where these codes are exported to and how important these are to UK's overall trade. 

***

 ### 1. Upload trade and commoidty code input files

In [1]:
# import data and libraries:

# upload trade data
import pandas as pd
import numpy as np

!pip install openpyxl

# chnage from scientific notation 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

trade1 = pd.read_excel("../hs_2022_exports/data/20210319_CRTA_trade_countries_A-L.xlsx", "Trade Flows")

trade2 = pd.read_excel("../hs_2022_exports/data/20210319_CRTA_trade_countries_M-Z.xlsx", "Trade Flows")

print(trade1.shape, trade2.shape)

Looking in indexes: https://s3-eu-west-2.amazonaws.com/mirrors.notebook.uktrade.io/pypi/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 105.9 MB/s eta 0:00:00
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
(920864, 7) (814795, 7)


In [2]:
# input data are two large import excel files with same data. 
# bind together:

trade = pd.concat([trade1, trade2])
trade.columns = trade.columns.str.lower().str.replace(" ","_") # clean column names
trade.dtypes

year                  int64
flow                 object
commodity_code       object
country_code         object
country_name         object
value_gbp             int64
suppression_notes    object
dtype: object

500+codes which are affected by HS chnages are uploaded as seperate input file

In [3]:
# upload HS codes sheet 1

hs_codes1 = pd.read_excel("../hs_2022_exports/data/HS 2022 codes.xlsx", sheet_name = "DeletedCodes", header = None)
hs_codes1 = hs_codes1.rename(columns={hs_codes1.columns[0]: 'commodity_code'})
hs_codes1["type"] = "deleted" # create identifier for code type
hs_codes1.shape

(349, 2)

In [4]:
# upload HS codes sheet 2

hs_codes2 = pd.read_excel("../hs_2022_exports/data/HS 2022 codes.xlsx", sheet_name = "ReusedCodes", header = None)
hs_codes2 = hs_codes2.rename(columns={hs_codes2.columns[0]: 'commodity_code'})
hs_codes2["type"] = "reused"
hs_codes2.shape

(173, 2)

In [5]:
# compile single commodity code list:
hs_codes = pd.concat([hs_codes1, hs_codes2], axis=0)
hs_codes.columns = hs_codes.columns.str.lower()
hs_codes

,commodity_code,type
0,0305 10 00,deleted
1,0307 21 00,deleted
2,0307 29 00,deleted
3,0403 10 11,deleted
4,0403 10 13,deleted
...,...,...
168,9022 29 00,reused
169,9404 90 10,reused
170,9404 90 90,reused
171,9405 99 00,reused


In [6]:
# clean commodity_code string:

hs_codes["commodity_code"] =  hs_codes["commodity_code"].str.slice(0,4) + hs_codes["commodity_code"].str.slice(5,7) + hs_codes["commodity_code"].str.slice(8,10)
hs_codes

,commodity_code,type
0,03051000,deleted
1,03072100,deleted
2,03072900,deleted
3,04031011,deleted
4,04031013,deleted
...,...,...
168,90222900,reused
169,94049010,reused
170,94049090,reused
171,94059900,reused


### 2. Upload correlation tables

In order to accurately create a time series the commodity codes need to be in the same nomenclature (i.e 8 digits). Codes can chnage year to year "concordance" tracks these changes. Codes need to be checked to see what nomenclature year they are (i.e. 2018,2019, 2020 or 2021). Once identifed they need changing to the same index year (2021). Imports data is in 2021. 

In [ ]:
# upload data:
# conversion tables 2018:2020, 2019:2020 and 2020:2021. 

cn8_2018 = pd.read_excel("../hs_2022_exports/data/CN8_2020_conversion_table.xlsx", sheet_name = "2018",dtype='object')
cn8_2019 = pd.read_excel("../hs_2022_exports/data/CN8_2020_conversion_table.xlsx", sheet_name = "2019",dtype='object')
cn8_2021 = pd.read_excel("../hs_2022_exports/data/CN8_2021_conversion_table.xlsx", dtype = "object")


print(cn8_2018.dtypes, cn8_2019.dtypes, cn8_2021.dtypes)

In [ ]:
## convert  to string and calculate string length. len() function wouldn't work without map(str)
cn8_2018["code_length"] = cn8_2018["CN8_2018"].map(str).apply(len)
cn8_2019["code_length"] = cn8_2019["CN8_2019"].map(str).apply(len)
cn8_2021["code_length"] = cn8_2021["CN8_2020"].map(str).apply(len)

In [ ]:
cn8_2018

Correlation files due to excel conversions have 7 digit codes instead of 8 - where excel has converted to code to numerical thus removing the "0". This needs correcting:

In [ ]:
## concatonate commodty codes with "0" if length is 7. 
# 2018
cn8_2018["CN8_2018"] = np.where(cn8_2018["code_length"] == 7, "0" + cn8_2018["CN8_2018"].map(str), cn8_2018["CN8_2018"])
cn8_2018["CN8_2020"] = np.where(cn8_2018["code_length"] == 7, "0" + cn8_2018["CN8_2020"].map(str), cn8_2018["CN8_2020"])

#2019
cn8_2019["CN8_2019"] = np.where(cn8_2019["code_length"] == 7, "0" + cn8_2019["CN8_2019"].map(str), cn8_2019["CN8_2019"])
cn8_2019["CN8_2020"] = np.where(cn8_2019["code_length"] == 7, "0" + cn8_2019["CN8_2020"].map(str), cn8_2019["CN8_2020"])

#2021

cn8_2021["CN8_2020"] = np.where(cn8_2021["code_length"] == 7, "0" + cn8_2021["CN8_2020"].map(str), cn8_2021["CN8_2020"])
cn8_2021["CN8_2021"] = np.where(cn8_2021["code_length"] == 7, "0" + cn8_2021["CN8_2021"].map(str), cn8_2021["CN8_2021"])

### 3. Concordance Function:

Function to convert commodity code nomenclature to user specified year using correlation tables.

In [7]:
def concordFunction(df, yr1, yr2):
  
  if(yr1 >= 2018 & yr1 < yr2 & yr2 <= 2020):
   
    yr2 = str(yr2)
    text = "../hs_2022_exports/data/CN8_"+yr2+"_conversion_table.xlsx"
        
    cn8 = pd.read_excel(text, sheet_name = str(yr1),dtype='object')
    cn8.columns = cn8.columns.str.lower()
    # convert 7 digit commodity codes to 8 digit by concat. with "0"
    
    cn8["code_length"] = cn8[cn8.columns[0]].map(str).apply(len)
    cn8[cn8.columns[0]] =  np.where(cn8["code_length"] == 7, "0" + cn8[cn8.columns[0]].map(str), cn8[cn8.columns[0]]).astype(str)
    cn8[cn8.columns[1]] =  np.where(cn8["code_length"] == 7, "0" + cn8[cn8.columns[1]].map(str), cn8[cn8.columns[1]]).astype(str)
    
    
    # rename code names (old code, new code)
    
    cn8.rename(columns={cn8.columns[0]: 'current_code', cn8.columns[1]: 'new_code'}, inplace = True)
    
    # match together df and concordance code list using the yr1 code column 
    # (yr1 is the first column in the concordance files)
    
    cn8_concord = pd.merge(df,cn8, left_on = "commodity_code", right_on = cn8[cn8.columns[0]], how = "left")
    
    cn8_concord2 = cn8_concord.loc[(cn8_concord["conversion"] == 1)]

    nrow = cn8_concord2.shape[0]

    print(str(nrow), " Codes to concord:")

    cn8_concord["commodity_code_original"] = cn8_concord["commodity_code"]
    
    # update code if conversion flagged and is there is no match for a specific code, set code to original:
    cn8_concord["commodity_code"] = np.where(cn8_concord["conversion"] == 1, cn8_concord["new_code"], 
                                    np.where(cn8_concord["new_code"].isna(),cn8_concord["commodity_code_original"],cn8_concord["current_code"]))
    
    return(cn8_concord)
   
  elif((yr1 == 2020) & (yr2 == 2021)): 
   
    text = "../hs_2022_exports/data/CN8_2021_conversion_table.xlsx"
    cn8 = pd.read_excel(text, sheet_name = str(yr1),dtype='object')
    cn8.columns = cn8.columns.str.lower()
    
    cn8["code_length"] = cn8[cn8.columns[0]].map(str).apply(len)
    cn8[cn8.columns[0]] =  np.where(cn8["code_length"] == 7, "0" + cn8[cn8.columns[0]].map(str), cn8[cn8.columns[0]]).astype(str)
    cn8[cn8.columns[1]] =  np.where(cn8["code_length"] == 7, "0" + cn8[cn8.columns[1]].map(str), cn8[cn8.columns[1]]).astype(str)
    
    cn8.rename(columns={cn8.columns[0]: 'current_code', cn8.columns[1]: 'new_code'}, inplace = True)
    
    # match together df and concordance code list using the yr1 code column 
    # (yr1 is the first column in the concordance files)
    
    cn8_concord = pd.merge(df,cn8, left_on = "commodity_code", right_on = cn8[cn8.columns[0]], how = "left")
    
    cn8_concord2 = cn8_concord.loc[(cn8_concord["conversion"] == 1)]

    nrow = cn8_concord2.shape[0]

    print(str(nrow), " Codes to concord:")

    cn8_concord["commodity_code_original"] = cn8_concord["commodity_code"]
    
    cn8_concord["commodity_code"] = np.where(cn8_concord["conversion"] == 1, cn8_concord["new_code"], 
                                    np.where(cn8_concord["new_code"].isna(),cn8_concord["commodity_code_original"],cn8_concord["current_code"]))
    
    return(cn8_concord)
    
  else: 
     print("Error please input different years or valid dataframe")
  
  
  # after matching codes together - identify how many codes are different

In [9]:
# convert commodity codes to 2020 nomenclature. 

hs_codes20 = concordFunction(hs_codes,2018,2020)
hs_codes20.drop(["current_code", 'new_code',"conversion","code_length"], axis=1, inplace=True)
hs_codes20

7  Codes to concord:


,commodity_code,type,commodity_code_original
0,03051000,deleted,03051000
1,03072100,deleted,03072100
2,03072900,deleted,03072900
3,04031011,deleted,04031011
4,04031013,deleted,04031013
...,...,...,...
517,90222900,reused,90222900
518,94049010,reused,94049010
519,94049090,reused,94049090
520,94059900,reused,94059900


In [10]:
# check for NAs:
na_df = hs_codes20[hs_codes20['commodity_code'].isnull()]
na_df

,commodity_code,type,commodity_code_original


In [11]:
# concord commodity codes to 2021 nomenclature (to match against trade data which is 2021 nomenclature)
hs_codes21 = concordFunction(hs_codes20,2020,2021)
hs_codes21

1  Codes to concord:


,commodity_code,type,commodity_code_original,current_code,new_code,conversion,code_length
0,03051000,deleted,03051000,03051000,03051000,0,7
1,03072100,deleted,03072100,03072100,03072100,0,7
2,03072900,deleted,03072900,03072900,03072900,0,7
3,04031011,deleted,04031011,04031011,04031011,0,7
4,04031013,deleted,04031013,04031013,04031013,0,7
...,...,...,...,...,...,...,...
517,90222900,reused,90222900,90222900,90222900,0,8
518,94049010,reused,94049010,94049010,94049010,0,8
519,94049090,reused,94049090,94049090,94049090,0,8
520,94059900,reused,94059900,94059900,94059900,0,8


In [12]:
# check for NAs:
hs_codes21.isnull().sum()

commodity_code             0
type                       0
commodity_code_original    0
current_code               0
new_code                   0
conversion                 0
code_length                0
dtype: int64

Now all codes have been concorded to 2021 nomenclature - this dataset can now be matched against the trade data for HS 2022 export analysis. 

### 4. Determine code matching between trade data

Now the HS codes input is in 2021 nomenclature - determine if all codes are in trade data and investigate

In [13]:
# create unique df of trade commodity codes
trade_cn8 = trade[["commodity_code"]].drop_duplicates()
trade_cn8["hs2"] = trade_cn8["commodity_code"].str.slice(0,2)
print(trade_cn8.shape,trade_cn8.dtypes)

(9439, 2) commodity_code    object
hs2               object
dtype: object


In [14]:
# match HS codes to distinct cn8 trade list and identify any codes which don't match:

hs_codes_na = pd.merge(hs_codes21,trade_cn8, on = "commodity_code", how = "left")
hs_codes_na.isnull().sum()

commodity_code              0
type                        0
commodity_code_original     0
current_code                0
new_code                    0
conversion                  0
code_length                 0
hs2                        15
dtype: int64

15 codes don't match; investigate further:

In [15]:
hs_codes_na = hs_codes_na[hs_codes_na['hs2'].isnull()]
hs_codes_na

,commodity_code,type,commodity_code_original,current_code,new_code,conversion,code_length,hs2
79,36030020,deleted,36030020,36030020,36030020,0,8,NaN
80,36030030,deleted,36030030,36030030,36030030,0,8,NaN
81,36030040,deleted,36030040,36030040,36030040,0,8,NaN
82,36030050,deleted,36030050,36030050,36030050,0,8,NaN
83,36030060,deleted,36030060,36030060,36030060,0,8,NaN
84,36030080,deleted,36030080,36030080,36030080,0,8,NaN
88,38247200,deleted,38247200,38247200,38247200,0,8,NaN
92,38247600,deleted,38247600,38247600,38247600,0,8,NaN
93,38247700,deleted,38247700,38247700,38247700,0,8,NaN
290,88039010,deleted,88039010,88039010,88039010,0,8,NaN


**Conclusion**: investigating trade data the above codes do not exist - this indicates UK does not trade in these commodity codes. No issue with HS code list. 

##### Filter trade data based on HS codes list:

In [16]:
# unique hs list as an array:

cd = pd.unique(hs_codes21["commodity_code"])
cd

trade_hs = trade[trade["commodity_code"].isin(cd)]      
trade_hs.shape

# QA check - cn8 distinct ocunt should equal count of HS codes list. 
trade_hs_cn8 = trade_hs[["commodity_code"]].drop_duplicates()
trade_hs_cn8

,commodity_code
9,21069092
27,30021900
28,30022090
34,30062000
37,32041700
...,...
503659,29313700
504139,38247300
99094,38247500
176498,81073000


507 rows (15 NA codes from a total of 522) - row numbers matches - no issue. 

In [18]:
# filter for exports only before aggregating to cn8 level
trade_hs_exports = trade_hs[(trade_hs["flow"] == "Exports") & (trade_hs["country_name"] != "World total")]
trade_hs_exports = trade_hs_exports.drop("suppression_notes",1)
trade_hs_exports

,year,flow,commodity_code,country_code,country_name,value_gbp
9,2018,Exports,21069092,AF,Afghanistan,40949
27,2018,Exports,30021900,AF,Afghanistan,5005
28,2018,Exports,30022090,AF,Afghanistan,55964
34,2018,Exports,30062000,AF,Afghanistan,8147
37,2018,Exports,32041700,AF,Afghanistan,3026
...,...,...,...,...,...,...
760026,2020,Exports,94039090,ZW,Zimbabwe,9715
760028,2020,Exports,94049090,ZW,Zimbabwe,2571
760029,2020,Exports,94054039,ZW,Zimbabwe,1645
760030,2020,Exports,94054099,ZW,Zimbabwe,3000


In [19]:
# distinct number of codes with export data:
trade_hs_exports_dist = trade_hs_exports[["commodity_code"]].drop_duplicates()
trade_hs_exports_dist.shape

(500, 1)

### 5. Calculate highest exported commodity codes:

Want to calculate average trade values between 2018-2020 and identify the highest exported

In [20]:
# aggregate dataset to year and commodity code:
trade_hs_exports_cn8 = trade_hs_exports.groupby(["year", "commodity_code"], as_index = False).agg({"value_gbp": "sum"})
trade_hs_exports_cn8

,year,commodity_code,value_gbp
0,2018,02089098,3983360
1,2018,02109939,6280296
2,2018,02109990,113356
3,2018,03051000,478268
4,2018,03061990,2727808
...,...,...,...
1482,2020,97019000,104712772
1483,2020,97020000,149407016
1484,2020,97030000,626323077
1485,2020,97050000,268717431


In [21]:
# compile to wide format:
trade_hs_exports_cn8_wide = trade_hs_exports_cn8.pivot(index='commodity_code', columns='year', values='value_gbp').copy()
trade_hs_exports_cn8_wide = trade_hs_exports_cn8_wide.reset_index() ## without this - commodity codecolumn isn't created. 
trade_hs_exports_cn8_wide.head(5)

year,commodity_code,2018,2019,2020
0,02089098,3983360.00000,4985347.00000,2759039.00000
1,02109939,6280296.00000,12234417.00000,3510066.00000
2,02109990,113356.00000,95444.00000,186680.00000
3,03051000,478268.00000,1322410.00000,493910.00000
4,03061990,2727808.00000,1193404.00000,803821.00000


In [22]:
# drop year as index column:
trade_hs_exports_cn8_wide = trade_hs_exports_cn8_wide.rename_axis(None, axis=1)

In [23]:
## average values across columns and rank:
trade_hs_exports_cn8_wide["average"] = np.mean(trade_hs_exports_cn8_wide, axis = 1)
trade_hs_exports_cn8_wide["rank"] = trade_hs_exports_cn8_wide["average"].rank(ascending =False)
trade_hs_exports_cn8_wide.sort_values("rank", inplace = True)
trade_hs_exports_cn8_wide.head(10)

,commodity_code,2018,2019,2020,average,rank
435,88033000,11482672185.00000,11506361863.00000,7570132700.00000,10186388916.00000,1.00000
494,97011000,3434108247.00000,6734085186.00000,3530362249.00000,4566185227.33333,2.00000
108,30022090,3209116957.00000,2672221518.00000,760330741.00000,2213889738.66667,3.00000
106,30021500,2367162970.00000,1814290997.00000,1840476646.00000,2007310204.33333,4.00000
130,38220000,1385984449.00000,1539289452.00000,2138591916.00000,1687955272.33333,5.00000
378,85171200,934114678.00000,1185407862.00000,899651703.00000,1006391414.33333,6.00000
432,88024000,952581554.00000,678905840.00000,997424966.00000,876304120.00000,7.00000
440,89039210,686239757.00000,1320468343.00000,506711477.00000,837806525.66667,8.00000
497,97030000,756497627.00000,1073129042.00000,626323077.00000,818649915.33333,9.00000
499,97060000,737435765.00000,1012108132.00000,509674508.00000,753072801.66667,10.00000


In [24]:
## calculate code exports as a proportion of these select HS codes.
total_hs_exports = np.sum(trade_hs_exports_cn8_wide["average"])
trade_hs_exports_cn8_wide["proportion"] = trade_hs_exports_cn8_wide["average"] / total_hs_exports
trade_hs_exports_cn8_wide

,commodity_code,2018,2019,2020,average,rank,proportion
435,88033000,11482672185.00000,11506361863.00000,7570132700.00000,10186388916.00000,1.00000,0.22729
494,97011000,3434108247.00000,6734085186.00000,3530362249.00000,4566185227.33333,2.00000,0.10189
108,30022090,3209116957.00000,2672221518.00000,760330741.00000,2213889738.66667,3.00000,0.04940
106,30021500,2367162970.00000,1814290997.00000,1840476646.00000,2007310204.33333,4.00000,0.04479
130,38220000,1385984449.00000,1539289452.00000,2138591916.00000,1687955272.33333,5.00000,0.03766
...,...,...,...,...,...,...,...
163,44071910,55.00000,4614.00000,2153.00000,2274.00000,496.00000,0.00000
423,87043210,NaN,NaN,1372.00000,1372.00000,497.00000,0.00000
86,29313100,63.00000,1044.00000,1121.00000,742.66667,498.00000,0.00000
131,38247100,983.00000,321.00000,877.00000,727.00000,499.00000,0.00000


In [ ]:
# save individual output file
trade_hs_exports_cn8_wide.to_excel('../hs_2022_exports/outputs/hs_2022_top_exported_codes.xlsx', index = False)

### 6. Calculate largest countries UK exports to

Aggregate HS trade export data to country level and create ranking. 

In [25]:
## group by trade export data
# filter for exports only before aggregating to cn8 level
trade_hs_exports_country = trade_hs_exports.groupby(["year","country_code","country_name"], as_index = False).agg({"value_gbp": "sum"})
trade_hs_exports_country

,year,country_code,country_name,value_gbp
0,2018,'NA,Namibia,16234078
1,2018,AD,Andorra,1002882
2,2018,AE,United Arab Emirates,534548302
3,2018,AF,Afghanistan,4533907
4,2018,AG,Antigua and Barbuda,4025016
...,...,...,...,...
672,2020,XS,Serbia,19431113
673,2020,YE,Yemen,3373953
674,2020,ZA,South Africa,191037358
675,2020,ZM,Zambia,17794019


In [26]:
# pivot dataframe into wide format
trade_hs_exports_country = trade_hs_exports_country.pivot(index = ['country_code','country_name'], columns = 'year', values = 'value_gbp')
trade_hs_exports_country = trade_hs_exports_country.reset_index()
trade_hs_exports_country

year,country_code,country_name,2018,2019,2020
0,'NA,Namibia,16234078.00000,16679891.00000,18257642.00000
1,AD,Andorra,1002882.00000,8775371.00000,3416914.00000
2,AE,United Arab Emirates,534548302.00000,480405373.00000,452276418.00000
3,AF,Afghanistan,4533907.00000,2258102.00000,3837667.00000
4,AG,Antigua and Barbuda,4025016.00000,2072217.00000,8329676.00000
...,...,...,...,...,...
227,XS,Serbia,24687643.00000,19495125.00000,19431113.00000
228,YE,Yemen,2276899.00000,3955546.00000,3373953.00000
229,ZA,South Africa,197261675.00000,168207144.00000,191037358.00000
230,ZM,Zambia,30490306.00000,24183703.00000,17794019.00000


In [27]:
## average value and rank columns:
trade_hs_exports_country["average"] = np.mean(trade_hs_exports_country.iloc[:,[2,3,4]], axis = 1)
trade_hs_exports_country

year,country_code,country_name,2018,2019,2020,average
0,'NA,Namibia,16234078.00000,16679891.00000,18257642.00000,17057203.66667
1,AD,Andorra,1002882.00000,8775371.00000,3416914.00000,4398389.00000
2,AE,United Arab Emirates,534548302.00000,480405373.00000,452276418.00000,489076697.66667
3,AF,Afghanistan,4533907.00000,2258102.00000,3837667.00000,3543225.33333
4,AG,Antigua and Barbuda,4025016.00000,2072217.00000,8329676.00000,4808969.66667
...,...,...,...,...,...,...
227,XS,Serbia,24687643.00000,19495125.00000,19431113.00000,21204627.00000
228,YE,Yemen,2276899.00000,3955546.00000,3373953.00000,3202132.66667
229,ZA,South Africa,197261675.00000,168207144.00000,191037358.00000,185502059.00000
230,ZM,Zambia,30490306.00000,24183703.00000,17794019.00000,24156009.33333


EU countries need aggregating together. 
 
EU to be treated as one block - as an outcome of analysis is to help determine which tariff schedules to look through at the WTO. EU is a single entity at the WTO.

In [28]:
# upload EU code list
eu_codes = pd.read_excel("../hs_2022_exports/data/EUcodes.xlsx")
eu_codes = eu_codes[["eu_country", "iso_code"]]

In [29]:
# check shape before merge:
trade_hs_exports_country.shape

(232, 6)

In [30]:
## match EU codes to trade country df and create new country name column
trade_hs_exports_country = pd.merge(trade_hs_exports_country, eu_codes, left_on = "country_code", right_on = "iso_code", how = "left")
trade_hs_exports_country["country_name_eu"] = np.where(trade_hs_exports_country["eu_country"].isnull() == True, trade_hs_exports_country["country_name"],"EU")
trade_hs_exports_country["country_code"] =  np.where(trade_hs_exports_country["eu_country"].isnull() == True, trade_hs_exports_country["country_code"],"EU")
trade_hs_exports_country.head(20)

,country_code,country_name,2018,2019,2020,average,eu_country,iso_code,country_name_eu
0,'NA,Namibia,16234078.00000,16679891.00000,18257642.00000,17057203.66667,NaN,NaN,Namibia
1,AD,Andorra,1002882.00000,8775371.00000,3416914.00000,4398389.00000,NaN,NaN,Andorra
2,AE,United Arab Emirates,534548302.00000,480405373.00000,452276418.00000,489076697.66667,NaN,NaN,United Arab Emirates
3,AF,Afghanistan,4533907.00000,2258102.00000,3837667.00000,3543225.33333,NaN,NaN,Afghanistan
4,AG,Antigua and Barbuda,4025016.00000,2072217.00000,8329676.00000,4808969.66667,NaN,NaN,Antigua and Barbuda
5,AI,Anguilla,72818.00000,114394.00000,444032.00000,210414.66667,NaN,NaN,Anguilla
6,AL,Albania,1117249.00000,2099820.00000,1924606.00000,1713891.66667,NaN,NaN,Albania
7,AM,Armenia,624305.00000,1015580.00000,799293.00000,813059.33333,NaN,NaN,Armenia
8,AO,Angola,12500270.00000,22149342.00000,17900614.00000,17516742.00000,NaN,NaN,Angola
9,AQ,Antarctica,20000.00000,646172.00000,372665.00000,346279.00000,NaN,NaN,Antarctica


In [31]:
# aggregate EU countries into one single row and create highest exported country rank:
trade_hs_exports_country2 = trade_hs_exports_country.groupby(["country_name_eu", "country_code"], as_index = False).sum()

trade_hs_exports_country2["rank"] = trade_hs_exports_country2["average"].rank(ascending =False)
trade_hs_exports_country2.sort_values("rank", inplace = True)

trade_hs_exports_country2["proportion"] = trade_hs_exports_country2["average"] / total_hs_exports
trade_hs_exports_country2

,country_name_eu,country_code,2018,2019,2020,average,rank,proportion
55,EU,EU,20773148768.00000,21320563864.00000,17025805725.00000,19706506119.00000,1.00000,0.43972
193,United States,US,10960631725.00000,13532517722.00000,10112506042.00000,11535218496.33333,2.00000,0.25739
175,Switzerland,CH,1264371538.00000,2967657461.00000,1579097661.00000,1937042220.00000,3.00000,0.04322
42,China,CN,1237491306.00000,1306429688.00000,1024145389.00000,1189355461.00000,4.00000,0.02654
81,Hong Kong,HK,1186493752.00000,1270423095.00000,798174089.00000,1085030312.00000,5.00000,0.02421
...,...,...,...,...,...,...,...,...
146,Pitcairn,PN,0.00000,3475.00000,5241.00000,4358.00000,202.00000,0.00000
114,Melilla,XL,0.00000,4045.00000,925.00000,2485.00000,203.00000,0.00000
43,Cocos Islands,CC,0.00000,2462.00000,0.00000,2462.00000,204.00000,0.00000
27,Bouvet Island,BV,3000.00000,0.00000,1200.00000,2100.00000,205.00000,0.00000


#### 7. Highest exported countries per cn8 codes dataframe

Create an output which highlights the highest exported commodity codes and the largest export market (country) per code.

This creates a commodity code/country aggregated level dataframe - based on the highest exported trade values on average between 2018-2020

In [32]:
trade_hs_exports_full = trade_hs_exports.groupby(["commodity_code", "country_name", "country_code"], as_index = False).agg({"value_gbp": "mean"})
trade_hs_exports_full

,commodity_code,country_name,country_code,value_gbp
0,02089098,Belgium,BE,55616.00000
1,02089098,Cyprus,CY,371.00000
2,02089098,Denmark,DK,129056.00000
3,02089098,Falklands Islands and dependencies,FK,11198.00000
4,02089098,France,FR,288591.33333
...,...,...,...,...
32970,97060000,United States,US,418653518.66667
32971,97060000,United States Virgin Islands,VI,9551.50000
32972,97060000,Uruguay,UY,14356.33333
32973,97060000,Uzbekistan,UZ,773851.00000


In [33]:
trade_hs_exports_full = trade_hs_exports.groupby(["commodity_code", "country_name", "country_code"], as_index = False).agg({"value_gbp": "mean"})
trade_hs_exports_full

,commodity_code,country_name,country_code,value_gbp
0,02089098,Belgium,BE,55616.00000
1,02089098,Cyprus,CY,371.00000
2,02089098,Denmark,DK,129056.00000
3,02089098,Falklands Islands and dependencies,FK,11198.00000
4,02089098,France,FR,288591.33333
...,...,...,...,...
32970,97060000,United States,US,418653518.66667
32971,97060000,United States Virgin Islands,VI,9551.50000
32972,97060000,Uruguay,UY,14356.33333
32973,97060000,Uzbekistan,UZ,773851.00000


In [34]:
# match commodity code ranking
trade_hs_exports_full2 = pd.merge(trade_hs_exports_full, trade_hs_exports_cn8_wide[["rank", "commodity_code"]], left_on = "commodity_code", right_on = "commodity_code", how = "left")
trade_hs_exports_full2

,commodity_code,country_name,country_code,value_gbp,rank
0,02089098,Belgium,BE,55616.00000,276.00000
1,02089098,Cyprus,CY,371.00000,276.00000
2,02089098,Denmark,DK,129056.00000,276.00000
3,02089098,Falklands Islands and dependencies,FK,11198.00000,276.00000
4,02089098,France,FR,288591.33333,276.00000
...,...,...,...,...,...
32970,97060000,United States,US,418653518.66667,10.00000
32971,97060000,United States Virgin Islands,VI,9551.50000,10.00000
32972,97060000,Uruguay,UY,14356.33333,10.00000
32973,97060000,Uzbekistan,UZ,773851.00000,10.00000


In [35]:
trade_hs_exports_full2.isnull().sum()

commodity_code    0
country_name      0
country_code      0
value_gbp         0
rank              0
dtype: int64

In [36]:
trade_hs_exports_full2 = trade_hs_exports_full2.sort_values(["rank"], ascending = True)
trade_hs_exports_full2.head(50)

,commodity_code,country_name,country_code,value_gbp,rank
26466,88033000,Anguilla,AI,10182.33333,1.00000
26475,88033000,Bahamas,BS,11296.00000,1.00000
26476,88033000,Bahrain,BH,3488998.00000,1.00000
26477,88033000,Bangladesh,BD,871270.00000,1.00000
26478,88033000,Barbados,BB,126040.00000,1.00000
26479,88033000,Belarus,BY,177940.66667,1.00000
26480,88033000,Belgium,BE,12154169.00000,1.00000
26481,88033000,Belize,BZ,3698.00000,1.00000
26547,88033000,Ireland,IE,16930996.66667,1.00000
26548,88033000,Israel,IL,16474442.00000,1.00000


Commodity code rank is in order - however df needs reordering to show top export markets at the top of the grouping:

In [37]:
trade_hs_exports_full2 = trade_hs_exports_full2.groupby(["rank"]).apply(lambda x: x.sort_values(["value_gbp"], ascending = False))
trade_hs_exports_full2

commodity_code   country_name country_code        value_gbp  \
rank                                                                          
1.00000   26530       88033000        Germany           DE 3072962245.00000   
          26525       88033000         France           FR 2991209962.33333   
          26644       88033000  United States           US 1734589553.00000   
          26493       88033000         Canada           CA  508471490.66667   
          26549       88033000          Italy           IT  205578830.00000   
...                        ...            ...          ...              ...   
499.00000 6710        38247100    Netherlands           NL         54.50000   
          6708        38247100        Ireland           IE         42.00000   
          6705        38247100         Cyprus           CY          9.00000   
          6713        38247100       Slovakia           SK          8.00000   
500.00000 2629        29033911         Cyprus           CY         22.00000   

                     rank  
rank                       
1.00000   26530   1.00000  
          26525   1.00000  
          26644   1.00000  
          26493   1.00000  
          26549   1.00000  
...                   ...  
499.00000 6710  499.00000  
          6708  499.00000  
          6705  499.00000  
          6713  499.00000  
500.00000 2629  500.00000  

[32975 rows x 5 columns]

In [40]:
trade_hs_exports_full3 = trade_hs_exports_full2.copy()

# create country export value proportion for total export of each commodity code:
trade_hs_exports_full3['country_pc'] =trade_hs_exports_full3['value_gbp'] / trade_hs_exports_full3.groupby('commodity_code')['value_gbp'].transform('sum')

In [41]:
trade_hs_exports_full3

commodity_code   country_name country_code        value_gbp  \
rank                                                                          
1.00000   26530       88033000        Germany           DE 3072962245.00000   
          26525       88033000         France           FR 2991209962.33333   
          26644       88033000  United States           US 1734589553.00000   
          26493       88033000         Canada           CA  508471490.66667   
          26549       88033000          Italy           IT  205578830.00000   
...                        ...            ...          ...              ...   
499.00000 6710        38247100    Netherlands           NL         54.50000   
          6708        38247100        Ireland           IE         42.00000   
          6705        38247100         Cyprus           CY          9.00000   
          6713        38247100       Slovakia           SK          8.00000   
500.00000 2629        29033911         Cyprus           CY         22.00000   

                     rank  country_pc  
rank                                   
1.00000   26530   1.00000     0.30165  
          26525   1.00000     0.29363  
          26644   1.00000     0.17027  
          26493   1.00000     0.04991  
          26549   1.00000     0.02018  
...                   ...         ...  
499.00000 6710  499.00000     0.04278  
          6708  499.00000     0.03297  
          6705  499.00000     0.00706  
          6713  499.00000     0.00628  
500.00000 2629  500.00000     1.00000  

[32975 rows x 6 columns]

Export all main dataframes to single spreadsheet output for further analysis and policy consideration:

In [43]:
with pd.ExcelWriter('../hs_2022_exports/outputs/hs_2022_export_analysis.xlsx') as writer1:
    trade_hs_exports_cn8_wide.to_excel(writer1, sheet_name = 'CN8 rank', index = False)
    trade_hs_exports_full3.to_excel(writer1, sheet_name = "CN8 country data", index = False)
    trade_hs_exports_country2.to_excel(writer1, sheet_name = 'Country rank', index = False)    

#### 8. Format excel spreadhseet using openpyxl:

Automate the styling and formatting of the HS 2022 export analysis output using openpyxl:

In [44]:
from openpyxl import Workbook, load_workbook
from openpyxl.styles import numbers
from openpyxl.styles import Border, Side, PatternFill, Color, Font

In [55]:
wb = load_workbook('../hs_2022_exports/outputs/hs_2022_export_analysis.xlsx')
wb.sheetnames

['CN8 rank', 'CN8 country data', 'Country rank']

In [56]:
# worksheets:
ws = wb["CN8 rank"]
ws2 = wb["CN8 country data"]
ws3 = wb["Country rank"]

In [57]:
# iterate through each cell for values and format:

def convertNum(ws,startCol,endCol, pcFlag, pcCol):
    print(ws)
    
    if(pcFlag == "Y"):
    
      for col in range(startCol, endCol):
            for row in range(2, ws.max_row+1):
                ws.cell(column=col, row=row).number_format ='£#,##0' 
    
      for row in range(2,ws.max_row+1):
        ws.cell(column=pcCol, row=row).number_format = numbers.FORMAT_PERCENTAGE_00 # pre-built percentage format for 2.dp
        
    else:
    
       for col in range(startCol, endCol):
            for row in range(2, ws.max_row+1):
                ws.cell(column=col, row=row).number_format ='£#,##0' 
        
                

In [58]:
# convert number format across each worksheet:
convertNum(ws,2,6,"Y",7)
convertNum(ws2,4,5,"Y",6)
convertNum(ws3,3,7,"Y",8)

<Worksheet "CN8 rank">
<Worksheet "CN8 country data">
<Worksheet "Country rank">


In [59]:
# function for border styles for worksheet input:
def borderFunc(ws):
    print(ws)
    
    # header fill style:
    headerFill = PatternFill(start_color='4F81BD',
                   end_color='4F81BD',
                   fill_type='solid')
    
    
    # apply header styles:
    for col in range(ws.min_column, ws.max_column+1):
            ws.cell(column=col, row=1).border = Border(top = Side(border_style='thin', color='FF000000'),    
                                               right = Side(border_style='thin', color='FF000000'), 
                                               bottom = Side(border_style='thin', color='FF000000'),
                                               left = Side(border_style='thin', color='FF000000'))
            
            ws.cell(column=col, row=1).fill = headerFill
            ws.cell(column=col, row=1).font = Font(color = '00FFFFFF', bold = True)
            
    # table border style:     
    for col in range(ws.min_column, ws.max_column+1):
        for row in range(2, ws.max_row+1):
            ws.cell(column=col, row=row).border = Border(top = Side(border_style='thin', color='4F81BD'),  
                                              bottom = Side(border_style='thin', color='4F81BD'))

In [66]:
# Apply border style across each worksheet:
borderFunc(ws)
borderFunc(ws2)
borderFunc(ws3)

<Worksheet "CN8 rank">
<Worksheet "CN8 country data">
<Worksheet "Country rank">


In [64]:
# Set column widths across each sheet:

#sheet 1
ws.column_dimensions['A'].width = 15
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 20
ws.column_dimensions['D'].width = 20
ws.column_dimensions['E'].width = 20
ws.column_dimensions['G'].width = 12

#sheet 2

ws2.column_dimensions['A'].width = 15
ws2.column_dimensions['B'].width = 20
ws2.column_dimensions['C'].width = 12
ws2.column_dimensions['D'].width = 20
ws2.column_dimensions['F'].width = 15

#sheet 3

ws3.column_dimensions['A'].width = 15
ws3.column_dimensions['B'].width = 12
ws3.column_dimensions['C'].width = 20
ws3.column_dimensions['D'].width = 15
ws3.column_dimensions['E'].width = 20
ws3.column_dimensions['F'].width = 20
ws3.column_dimensions['H'].width = 15



In [65]:
wb.save("../hs_2022_exports/outputs/hs_2022_export_analysis_pyxl.xlsx")

End.